# Análisis exploratorio y descripción general de los datos a usar (RawData)

## 0. Introducción

(Añadir descripción de cómo usar DataFiltering)

## 1. Cargar librerías

In [161]:
import pandas as pd
import numpy as np

import functions as fn

## 2. Leer las direcciones de RawData

In [117]:
fal_url = 'RawData/fallecidos_covid.csv'
vac_url = 'RawData/TB_VACUNACION_COVID19.csv'

## 3. Cargar el dataset de fallecidos por COVID-19

### 3.1. Cargamos una parte del dataset

In [166]:
df_fal = fn.complete_largeCSV_file(fal_url, ';')

Read csv with dask:  0.2541205883026123 sec
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201527 entries, 0 to 201526
Columns: 10 entries, FECHA_CORTE to id_persona
dtypes: category(8), float64(1), int64(1)
memory usage: 5.6 MB


### 3.2. Información general del dataset

In [167]:
df_fal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201527 entries, 0 to 201526
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype   
---  ------               --------------   -----   
 0   FECHA_CORTE          201527 non-null  category
 1   FECHA_FALLECIMIENTO  201527 non-null  int64   
 2   EDAD_DECLARADA       201527 non-null  category
 3   SEXO                 201527 non-null  category
 4   CLASIFICACION_DEF    201527 non-null  category
 5   DEPARTAMENTO         201527 non-null  category
 6   PROVINCIA            201522 non-null  category
 7   DISTRITO             201522 non-null  category
 8   UBIGEO               201527 non-null  category
 9   id_persona           198129 non-null  float64 
dtypes: category(8), float64(1), int64(1)
memory usage: 5.4 MB


### 3.3. Porcentaje de datos perdidos de cada variable

In [164]:
fn.missing_values(df_fal)

,index,percent_missing
0,FECHA_CORTE,0.000000
1,FECHA_FALLECIMIENTO,0.000000
2,EDAD_DECLARADA,0.000000
3,SEXO,0.000000
4,CLASIFICACION_DEF,0.000000
5,DEPARTAMENTO,0.000000
6,PROVINCIA,0.002481
7,DISTRITO,0.002481
8,UBIGEO,0.000000
9,id_persona,1.686126


## 4. Analizar cada variable del dataset de fallecidos

### 4.1. FECHA_CORTE

In [173]:
fal_cor = df_fal['FECHA_CORTE'].value_counts().sort_index()
print('Fecha de corte:'), print(fal_cor.to_string(header=False))
del fal_cor

Fecha de corte:
20211208    201527


Todos los valores son iguales. La misma fecha de corte para n número de casos, variable no necesaria.

### 4.2. FECHA_FALLECIMIENTO

In [169]:
# Fecha del primer fallecido reportado
date_a = pd.DataFrame({'date':[df_fal['FECHA_FALLECIMIENTO'].min()]})
date_a = pd.to_datetime(date_a['date'], format = '%Y%m%d').dt.date
print(f'Fecha del primer fallecido reportado: {date_a[0]}')
del date_a

# Fecha del último fallecido reportado
date_b = pd.DataFrame({'date':[df_fal['FECHA_FALLECIMIENTO'].max()]})
date_b = pd.to_datetime(date_b['date'], format = '%Y%m%d').dt.date
print(f'Fecha del último fallecido reportado: {date_b[0]}')
del date_b

Fecha del primer fallecido reportado: 2020-03-03
Fecha del último fallecido reportado: 2021-12-08


### 4.3. EDAD_DECLARADA

In [170]:
fal_edad = df_fal['EDAD_DECLARADA'].value_counts()

print('Edad (años): no_casos\nDesde la edad más común a la menos común.\n')
lst = []

for idx, edad in enumerate(fal_edad):
    index = fal_edad.index
    lst.append(f'{index[idx]}: {edad}')
    
chunks = [lst[x:x+20] for x in range(0, len(lst), 20)]
df = pd.DataFrame.from_records(chunks)
df = df.transpose()
print(df.to_string(index=False, header=False))

del fal_edad, chunks, df, lst

Edad (años): no_casos
Desde la edad más común a la menos común.

66: 5479 80: 4115 48: 2214 34: 640   1: 145   5: 40
67: 5379 58: 4011 89: 2069 33: 622  20: 109 102: 40
65: 5373 79: 3994 47: 2015 95: 557  19: 109   6: 38
68: 5317 57: 3986 90: 1850 32: 553 100: 103 103: 21
70: 5230 82: 3825 46: 1850 96: 454   18: 88 104: 16
72: 5174 81: 3795 45: 1767 31: 431    2: 83  106: 9
73: 5158 56: 3698 44: 1668 30: 425   17: 75  108: 6
69: 5136 55: 3679 43: 1500 29: 343   16: 75  105: 5
64: 5079 83: 3543 91: 1425 27: 311  101: 68  107: 3
74: 5053 54: 3422 42: 1346 28: 310    4: 63  113: 2
71: 5039 84: 3364 92: 1331 97: 306   15: 56  110: 2
63: 5039 85: 3188 41: 1259 26: 266    9: 52   -9: 1
61: 5019 53: 3103 40: 1090 25: 243   11: 52  117: 1
75: 4899 86: 2816 39: 1058 98: 243   14: 52  109: 1
62: 4885 52: 2797 93: 1002 24: 198    8: 52  114: 1
76: 4765 51: 2696  38: 973 23: 189   10: 50    None
60: 4578 87: 2638  37: 830 99: 189    3: 50    None
77: 4566 50: 2560  94: 752 21: 172    7: 45    None

Existen valores negativos, necesario correguir. Edad mínima 0 años y edad máxima 117 años.

### 4.4. SEXO

In [148]:
fal_sexo = df_fal['SEXO'].value_counts().sort_index()

for idx, sex in enumerate(fal_sexo):
    index = fal_sexo.index
    print(f'{index[idx]}: {sex}')

.: 2
FEMENINO: 73165
MASCULINO: 128360


Hay puntos en lugar de vacíos.

### 4.5. CLASIFICACION_DEF

In [149]:
fal_cla = df_fal['CLASIFICACION_DEF'].value_counts().sort_index()

for idx, clase in enumerate(fal_cla):
    index = fal_cla.index
    print(f'{index[idx]}: {clase}')

del fal_cla
# Se encuentran los 7 criterios de clasificación, corregir tíldes

Criterio SINADEF: 57725
Criterio clÃ­nico: 9814
Criterio investigaciÃ³n EpidemiolÃ³gica: 16085
Criterio nexo epidemiolÃ³gico: 2968
Criterio radiolÃ³gico: 6722
Criterio serolÃ³gico: 43942
Criterio virolÃ³gico: 64271


### 4.6. DEPARTAMENTO

In [150]:
fal_dep = df_fal['DEPARTAMENTO'].value_counts().sort_index()

print('Fallecidos por departamentos: \n')
for idx, dep in enumerate(fal_dep):
    index = fal_dep.index
    print(f'{index[idx]}: {dep}')
print(f'\nTotal fallecidos: {fal_dep.sum()}')

del fal_dep

Fallecidos por departamentos: 

AMAZONAS: 1273
ANCASH: 6728
APURIMAC: 1519
AREQUIPA: 9718
AYACUCHO: 2148
CAJAMARCA: 4149
CALLAO: 10055
CUSCO: 4812
HUANCAVELICA: 1176
HUANUCO: 2701
ICA: 8569
JUNIN: 7061
LA LIBERTAD: 10324
LAMBAYEQUE: 8642
LIMA: 89006
LORETO: 4266
MADRE DE DIOS: 772
MOQUEGUA: 1526
PASCO: 1046
PIURA: 12163
PUNO: 4228
SAN MARTIN: 3030
TACNA: 1970
TUMBES: 1586
UCAYALI: 3059

Total fallecidos: 201527


### 4.7. PROVINCIAS

In [151]:
fal_pro = df_fal['PROVINCIA'].value_counts().sort_index()
print(f'Igual que la variable departamento pero con {len(fal_pro)} provincias')
del fal_pro

Igual que la variable departamento pero con 196 provincias


### 4.8. DISTRITO

In [152]:
fal_dis = df_fal['DISTRITO'].value_counts().sort_index()
print(f'Igual que la variable departamento pero con {len(fal_dis)} distritos')
del fal_dis

Igual que la variable departamento pero con 1643 distritos


### 4.9. UBIGEO

In [153]:
fal_ubi = df_fal['UBIGEO'].value_counts().sort_index()
print(f'{len(fal_ubi)} códigos de UBIGEO, para localizar zonas')
del fal_ubi

1778 códigos de UBIGEO, para localizar zonas


### 4.10. id_persona

In [158]:
df_fal['id_persona'] = df_fal['id_persona'].fillna(0)
df_fal['id_persona'] = df_fal['id_persona'].apply(np.int64)

print('ID únicos para identificar a cada persona: \n')
print(df_fal['id_persona'].head(10))

ID únicos para identificar a cada persona: 

0    24833991
1    24761117
2    24767070
3    24751741
4    24671820
5    24709216
6    24709228
7    25670721
8    25670904
9    25671021
Name: id_persona, dtype: int64


## 5. Cargar el dataset de vacunados contra COVID-19

### 5.1. Cargamos una parte del dataset

In [40]:
df_vac = pd.read_csv(vac_url, sep=';', nrows= 2)
print("Dataset de todos los vacunados contra COVID-19 en Perú: \n")
print(df_vac.head())

Dataset de todos los vacunados contra COVID-19 en Perú: 

  id_persona,id_vacunados_covid19,fecha_vacunacion,id_eess,id_centro_vacunacion,id_vacuna,id_grupo_riesgo,dosis,edad
0       10789963,3575662,20210701,18536,62,5,54,2,73                                                                
1        9193576,3575918,20210703,4482,752,5,54,2,60                                                                


Todas las variables son numéricas.

In [4]:
df_vac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   id_persona            2 non-null      int64
 1   id_vacunados_covid19  2 non-null      int64
 2   fecha_vacunacion      2 non-null      int64
 3   id_eess               2 non-null      int64
 4   id_centro_vacunacion  2 non-null      int64
 5   id_vacuna             2 non-null      int64
 6   id_grupo_riesgo       2 non-null      int64
 7   dosis                 2 non-null      int64
 8   edad                  2 non-null      int64
dtypes: int64(9)
memory usage: 272.0 bytes
